In [1]:
import re #28/01/2023 works!
import pandas as pd
import numpy as np
import pdfplumber
import shutil
import os, glob

dir_remov = ["C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/",
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/", 
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/Transporeon_split_1",
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/", 
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/Chawan_Vopak_email/",
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/Chawan_Vopak_email/Send_Vopak/"]
for dir_remo in dir_remov:
    for file_remov in glob.glob(dir_remo +"*.pdf"):
        os.remove(file_remov)

#move file, overwrite if exists
list_of_files = glob.glob('C:/Users/john.tan/Downloads/*.pdf') # *.csv
latest_file = max(list_of_files, key=os.path.getctime) #get latest files
src_path = r"C:/Users/john.tan/Downloads/"
dest_path = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/"

def substring_after(s, delim):
    return s.partition(delim)[2]

latest_transport = substring_after(latest_file, "\\")
if os.path.isfile(os.path.join(dest_path, latest_transport)):
    os.remove(os.path.join(dest_path, latest_transport)) #remove file if exists
shutil.move(src_path+latest_transport, dest_path)


from PyPDF2 import PdfWriter, PdfReader #split pdf files
from PyPDF2 import PdfWriter, PdfReader
path_file = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transports_Number_50.pdf"
file_out = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/Transport"
inputpdf = PdfReader(open(path_file, "rb"))

for i in range(len(inputpdf.pages)):
    output = PdfWriter()
    output.add_page(inputpdf.pages[i])
    with open(file_out + "_%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)
        
substr = 'Transporeon-ID' #remove pdf w/o 'Transporeon-ID'
path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/"
for file in glob.glob(path_dir+"*.pdf"):
    file = str(file)
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    
    if text.find(substr) !=-1:
        path_from = file
        #print(file)
        path_dest = 'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split_1/'
        shutil.move(path_from, path_dest)
        pdf.close()

        
regexp = re.compile("Origin(.*)$", re.MULTILINE)
path_dir_1 = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split_1/"
path_in = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/" #move pdf to 'Chawan', 'Vopak'
for file in glob.glob(path_dir_1 +"*.pdf"):
    file = str(file)
    path_from = file
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    try:
        ori = regexp.search(text).group(1).split('SG-',1)[0].split()[1]
        if ori in ['CHAWAN', 'VOPAK']:
            shutil.move(path_from, path_in)
            pdf.close()
    except:
        pass

In [2]:
dir = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/" #regex rename pdf
regexp = re.compile(r'(?s)Order type (.*?)\n[^\S\r\n]*Shipping ', re.MULTILINE)
regexp_2 = re.compile(r'(?s)Shipping number (BOL) (.*?)\n[^\S\r\n]*Gross ', re.MULTILINE)
regexp_3 = re.compile(r'Shipping number (BOL) (\d+)', re.MULTILINE)

for file in os.listdir(dir):
    f = os.path.join(dir, file) # iterate over files in dir
    if os.path.isfile(f): # checking if it is a file
        pdf = pdfplumber.open(f)
        page = pdf.pages[0]
        text = page.extract_text()
        pdf.close()
        try:
            ship_num = regexp.search(text).group(1) #
            ship_num = ".".join(re.findall(r'\d+', ship_num))
            f1 = f.rpartition('/')[0] + '/Transport_' + ship_num +'.pdf'
            os.rename(f, f1)
        except:
            pass
        try:
            ship_num = regexp_2.search(text).group(1) #
            f1 = f.rpartition('/')[0] + '/Transport_' + ship_num +'.pdf'
            os.rename(f, f1)
        except:
            pass

In [4]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.serialization import AzureJSONEncoder
#from azure.ai.formrecognizer import AnalyzeResult
import json
import glob, os

endpoint = "https://...cognitiveservices.azure.com/"
key = ""

path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/"
path_destination = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/"
i = 1

document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key))

list_of_files = glob.glob(path_dir+'*.pdf')
for file in list_of_files:
    file = str(file)
    path_to_sample_documents = file
    with open(path_to_sample_documents, "rb") as f:
        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-document", document=f
        )
    result = poller.result()

    analyze_result_dict = result.to_dict()
    with open(path_destination + 'data' + "_" +str(i)+ ".json", 'w') as f:
        json.dump(analyze_result_dict, f, cls=AzureJSONEncoder)
    i += 1


In [8]:
import re, os, glob #parse json files 23/1/2023!
import pandas as pd
import numpy as np
import json
from re import search

path_dir = 'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/'
cols = ['Cust Ref', 'Order Remarks', 'Transport No', 'Coll Date', 'DEPOT_MV2', 'PRODUCT', 'LOADING_PLANT_MV3', 'DELIVERY', 'SAP Order No']
df_trans = pd.DataFrame(columns=cols)
#df_trans.columns = cols
lst_trans = []
i = 0

os.chdir(path_dir)
for file in glob.glob("*.json"):
    file = str(file)
    with open(path_dir+file) as file_object:
        data = json.load(file_object) #parse json file to Python obj

    z = data['content']

    trans_num = re.search(r'Transport No. (.*?) Delivery', z).group(1) #extract str between two strings
    #trans_ID = re.search(r'ID (.*?) Weight', z, flags=re.MULTILINE).group(1)
    #weight = re.search(r'Weight (.*?)kg', z).group(1) #
    cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', z) #MV3
    print(cust[0])
    try:
            sap_num = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*Order type', z)
            sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
            print(sap_num)
    except:
            pass
    
    if (cust[0] == 'SINGAPORE ADHESION PLANT' and (isinstance(sap_num, list) == True)):
        try:
                sap_adh = re.findall(r"(\d+)\nOrder type", z)[0]
                print('SAP1')
                print(sap_adh)
        except:
                sap_numbr = re.search('SAP Order No.\n(\d+)', z)
                if sap_numbr:
                    print('SAP2')
                    sap_adh = sap_numbr.group(1)
                    print(sap_adh)           
                
    if cust[0] == "CHAMPIONX SG SERVICE PTE LTD":
        sap_num = re.findall(r'(?s)SG\n(.*?)\n[^\S\r\n]*normalorder', z)[0]
        #sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
        print(sap_num)
    try:
        shipper = re.search(r'(.*?) Scheduling', z).group(1) #
        shipping_num = re.search('\nnormalorder\n(\d+)', z, re.IGNORECASE).group(1) ## get numbers after str

    except:
        shipper = re.findall(r'(?s)Origin\n(.*?)\n[^\S\r\n]*SG', z)[0]       
    try:
        prod_info = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*UTC', z) #
        first, *others, last = prod_info[0].strip().splitlines()
        prod_info = last
        #print(last)
    except:
        prod_info = re.findall(r'(?s)Product information\n(.*?)\n[^\S\r\n]*Loading', z) #
        
    if search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
        try:
            #deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]* ', z)
            deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z, re.MULTILINE) #
            deli_datex = deli_date[0].split(' ')[3]
            #print(deli_datex)
        except:
            deli_datex = re.findall(r'(?s)MY-(.*?)\n[^\S\r\n]*Measurements', z, re.MULTILINE) #
    
    if cust[0] == "MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)":
        try:
            sap_num = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*Order type', z)
            sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
            print(sap_num)
        except:
            sap_num = re.findall(r'(?s)SAP Order No.(.*?)\n[^\S\r\n]*Items', z)
            sap_num = re.findall(r'(?s)SG(.*?)\n[^\S\r\n]*Delivery No.', sap_num[0], re.MULTILINE)
            #print(sap_num[0].strip())
            first, second, *others = sap_num[0].strip().splitlines()
            sap_num = first
            print(sap_num)
        z_msd = z.replace('\n', '')    
        try:
            shipping_numMSD = re.search('SingaporeSG(\d+)', z_msd, re.IGNORECASE).group(1) #
        except:
            pass
        try:
            shipping_numMSD = re.search('\nSG\n(\d+)', z_msd, re.IGNORECASE).group(1)
        except:
            pass
        try:
            shipping_numMSD = shipping_numMSD[-15:]
            shipping_numsd = shipping_numMSD[0:10]
            print('z_msd '+shipping_numsd)
        except:
            pass
        
    if cust[0] == "SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG":
        try:
            regexp = re.compile(r'(?s)Order type (.*?)\n[^\S\r\n]*Shipping ', re.MULTILINE)
            shipping_num = regexp.search(z).group(1) #
            shipping_numseika = ".".join(re.findall(r'\d+', shipping_num))
        except: 
            pass
    
    po_num = re.findall(r'(?s)Purchase order No.\n(.*?)\n[^\S\r\n]*Gross Weight', z)
    cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', z) #MV3
    origin = re.findall(r'(?s)Origin\n(.*?)\n[^\S\r\n]*SG-', z) #MV2
    plant = re.findall(r'(?s)Division\n(.*?)\n[^\S\r\n]*Product', z)
    #vehicle = re.search(r'AP - (.*?) T', z).group(1)
    #vehicle = str(vehicle) + 'Truck'
    
    #deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]* ', z)
    deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z) #
    deli_date = deli_date[0].split(' ', 1)[0]
    if cust[0] == 'EXXONMOBIL CHEMICAL COMPANY':
        deli_date = deli_date.split('\n')[0]
        
    if cust[0] == 'RESPACK MANUFACTURING SDN BHD':
        p = re.compile('MY-([\S\s]*)\n', re.MULTILINE) #Respack
        deli_date = re.findall(p, respack)[0].split('\n')[1].split(' ')[0]
    try:
        shipping_num = re.findall(r'(?s)Order type\n(.*?)\n[^\S\r\n]*Shipping ', z)[0] #
        shipping_num = ".".join(re.findall(r'\d+', shipping_num))
    except:
        pass
    try:
        shipping_num = re.search('\nnormalorder\n(\d+)', z, re.IGNORECASE).group(1)
    except:
        pass
    prod_info = re.findall(r'(?s)Product information\n(.*?)\n[^\S\r\n]*Loading', z) #
    
    try:
        deadline = re.search(r'Deadline (.*?) ', z).group(1)
        tifu_patt = r'(?<=General Text)(\s.+)?\nAdditional'
        tifu = re.findall(tifu_patt, z, re.DOTALL)[0]
        tifu = os.linesep.join([s for s in tifu.splitlines() if s])
        first, *others = tifu.splitlines()
    except:
        pass  
    try:
        ord_rem = cust[0] + prod_info[0]
    except:
        pass
    if search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
        z_cx = z.replace('\n', '')
        try:
            prod_info =  re.search(r'Article description(.*?)Comment', z_cx).group(1) ##regex between 2 strings
            ord_rem = cust[0]+prod_info
        except:
            prod_info = re.search(r'Article short description(.*?)Article description', z_cx) #
    if search('BASF SOUTH EAST ASIA PTE LTD', cust[0]):
        z_basf = z.replace('\n', '')
        try:
            prod_info =  re.search(r'Article description(.*?)Comment', z_basf) ##regex between 2 strings
            print(prod_info)
            ord_rem = cust[0]
        except:
            prod_info = re.search(r'Article short description(.*?)Article description', z_basf) # 
            ord_rem = cust[0]
    #lst_trans.extend([shipping_num, ord_rem, cust, deli_date, weight])
    if origin[0] =='M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant':
        df_trans.loc[i, 'MV2'] = 'EXX004'
    else:
        df_trans.loc[i, 'MV2'] = 'EXX003'
    if cust[0] == 'THE POLYOLEFIN - PULAU AYER MERBAU':
        try:
            cust_1 = cust[1].split('\n')[-1].split(', SG')[0]
            df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust_1
        except:
            pass
        try:
            cust_1 = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z) #THE POLYOLEFIN - PULAU AYER MERBAU
            df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust_1
        except:
            pass
    else: 
        df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust[0]
    
    try:
        df_trans.loc[i, 'Cust Ref'] = shipping_num
    except:
        df_trans.loc[i, 'Cust Ref'] = second
    if cust[0] == "MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)":
        try:
            df_trans.loc[i, 'Cust Ref'] = shipping_numsd
        except:
            df_trans.loc[i, 'Cust Ref'] = shipping_num
    
    if cust[0] == "SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG":
        try:
            df_trans.loc[i, 'Cust Ref'] = shipping_numseika
        except:
            df_trans.loc[i, 'Cust Ref'] = shipping_num
    try:
        df_trans.loc[i, 'Order Remarks'] = ord_rem
    except:
        pass
    df_trans.loc[i, 'Transport No'] = trans_num
    df_trans.loc[i, 'Coll Date'] = deli_date
    df_trans.loc[i, 'DEPOT_MV2'] = origin[0]
    try:
        df_trans.loc[i, 'PRODUCT'] = prod_info[0]
    except:
        df_trans.loc[i, 'PRODUCT'] = last
    if search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
        df_trans.loc[i, 'PRODUCT'] = prod_info
    
    df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust[0]
    df_trans.loc[i, 'DELIVERY'] = deli_date
    try:
        df_trans.loc[i, 'SAP Order No'] = sap_num
    except:
        pass
    if (cust[0] == 'SINGAPORE ADHESION PLANT' and (isinstance(sap_num, list) == True)):
        try:
            df_trans.loc[i, 'SAP Order No'] = sap_adh
        except:
            pass
    try:
        if cust[1] =="CHAMPIONX SG\nSERVICE PTE LTD":
            df_trans.loc[i, 'DELIVERY'] = deli_datex
            df_trans.loc[i, 'Coll Date'] = deli_datex
            df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust[1].replace('\n', ' ')
            df_trans.loc[i, 'DEPOT_MV2'] = origin[0].replace('Destination\n', '')
    except: pass
    i += 1

df_trans

MITSUI ELASTOMERS SINGAPORE PTE LTD
12720815
MITSUI ELASTOMERS SINGAPORE PTE LTD
12720816
THE POLYOLEFIN - PULAU AYER MERBAU
12726160
MEGACHEM MANUFACTURING PTE LTD
5950321
CHAMPIONX SG SERVICE PTE LTD
5954952
CHAMPIONX SG SERVICE PTE LTD
5955180
CHAMPIONX SG SERVICE PTE LTD
5955534
CHAMPIONX SG SERVICE PTE LTD
5955536
CHAMPIONX SG SERVICE PTE LTD
5955532
CHAMPIONX SG SERVICE PTE LTD
5955533
CHAMPIONX SG SERVICE PTE LTD
5955535
BASF SOUTH EAST ASIA PTE LTD
5954173
None
CLP INTERNATIONAL PTE LTD
12737491
CLP INTERNATIONAL PTE LTD
12737490
MITSUI ELASTOMERS SINGAPORE PTE LTD
12737626
MITSUI ELASTOMERS SINGAPORE PTE LTD
12737627
EXXONMOBIL CHEMICAL COMPANY
4911873799
CHAMPIONX SG SERVICE PTE LTD
5955538
CHAMPIONX SG SERVICE PTE LTD
5955537
A.I.M. CHEMICAL INDUSTRIES PTE LTD
12739889
BASF SOUTH EAST ASIA PTE LTD
12734191
None
MITSUI ELASTOMERS SINGAPORE PTE LTD
12737629
BASF SOUTH EAST ASIA PTE LTD
12734187
None
MITSUI ELASTOMERS SINGAPORE PTE LTD
12737628
BASF SOUTH EAST ASIA PTE LTD
1273

,Cust Ref,Order Remarks,Transport No,Coll Date,DEPOT_MV2,PRODUCT,LOADING_PLANT_MV3,DELIVERY,SAP Order No,MV2
0,1003980889,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108314617,2023-01-27,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-01-27,12720815,EXX004
1,1003982292,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108324472,2023-01-31,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-01-31,12720816,EXX004
2,1003982293,THE POLYOLEFIN - PULAU AYER MERBAUEXXSOL HEXAN...,6108324476,2023-01-31,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,EXXSOL HEXANE SP AP VL,THE POLYOLEFIN - PULAU AYER MERBAU,2023-01-31,12726160,EXX003
3,1003982307,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108324475,2023-01-31,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,2023-01-31,5950321,EXX003
4,1003982696,CHAMPIONX SG SERVICE PTE LTDSOLVESSO 150 AP VL,6108328088,2023-02-01,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-01,5954952,EXX003
5,1003982714,CHAMPIONX SG SERVICE PTE LTDSOLVESSO 150 AP VL,6108328087,2023-02-01,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-01,5955180,EXX003
6,1003982715,CHAMPIONX SG SERVICE PTE LTDSOLVESSO 150 AP VL,6108328090,2023-02-02,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-02,5955534,EXX003
7,1003982733,CHAMPIONX SG SERVICE PTE LTDSOLVESSO 150 AP VL,6108328091,2023-02-03,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-03,5955536,EXX003
8,1003982739,CHAMPIONX SG SERVICE PTE LTDSOLVESSO 150 AP VL,6108328086,2023-02-01,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-01,5955532,EXX003
9,1003982760,CHAMPIONX SG SERVICE PTE LTDSOLVESSO 150 AP VL,6108328089,2023-02-02,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-02,5955533,EXX003


In [2]:
basf_dict = {'BASF SOUTH EAST ASIA PTE LTD': 'EXXAL 8 ROP VL'}
basf_dict_lst= list(basf_dict.keys())
df_trans['PRODUCT'] = np.where(df_trans.LOADING_PLANT_MV3==basf_dict_lst[0], 'EXXAL 8 ROP VL', df_trans.PRODUCT)
#df_trans['LOADING_PLANT_MV3'].isin(basf_dict_lst)
#df_trans

In [3]:
temp = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Template_Local.csv") #template for upload
tables = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Tables.csv") #tables for mapping

In [4]:
temp_upload = pd.concat([temp, df_trans], ignore_index=True)

temp_upload = pd.merge(left=temp_upload, right=tables, how='left') #merge df 
temp_upload = temp_upload.fillna(method='ffill')
#temp_upload = temp_upload.drop(['MV3'], axis=1)
temp_upload = temp_upload.rename(columns={'Code': 'MV3'}) #for upload to TMS

In [5]:
colss = ['Customer No', 'Order Remarks', 'Delivery Type No', 'ISO abbreviation', 'Cargo Type No', 'MV1', 'MV2', 'MV4', 'MV5', 'MV6',
         'Plant Code', 'Special Handling No', 'Destination', 'MV3', 'Delivery Type']
temp_upload_1 = temp_upload.drop(columns=colss, axis=1)
temp_upload_1 = temp_upload_1.rename(columns={'LOADING_PLANT_MV3': 'LOADING PLANT', 'DEPOT_MV2': 'DEPOT'})
colss_reord = ['Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT', 'LOADING PLANT', 'DELIVERY', 'SAP Order No']
temp_upload_1 = temp_upload_1[colss_reord] #to compare with master
temp_upload_1 = temp_upload_1.reset_index().drop(columns='index')
#temp_upload_1

In [6]:
temp_upload_1['Coll Date'] = pd.to_datetime(temp_upload_1['Coll Date']).dt.strftime('%d/%m/%Y') #format datetime
temp_upload_1['DELIVERY'] = pd.to_datetime(temp_upload_1['DELIVERY']).dt.strftime('%d/%m/%Y')

temp_upload_1['SAP Order No'] = pd.to_numeric(temp_upload_1['SAP Order No'])
temp_upload_1.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
temp_upload_1.index = np.arange(1, len(temp_upload_1)+1)
temp_upload_1

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
1,1003981694,6108320268,17/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,17/01/2023,5944859
2,1003981634,6108320286,17/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,17/01/2023,5944862
3,1003982307,6108324475,31/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,31/01/2023,5950321
4,1003981242,6108316731,16/01/2023,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,EXXAL 8 ROP VL,BASF SOUTH EAST ASIA PTE LTD,16/01/2023,5954173
5,1003982696,6108328088,01/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,01/02/2023,5954952
6,1003982714,6108328087,01/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,01/02/2023,5955180
7,1003982739,6108328086,01/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,01/02/2023,5955532
8,1003982760,6108328089,02/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,02/02/2023,5955533
9,1003982715,6108328090,02/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,02/02/2023,5955534
10,1003982780,6108328092,03/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,03/02/2023,5955535


In [11]:
columns = ['NO', 'SH Reference', 'Job Type', 'Shipper', 'Permit', 'ETA Date', 'QTY', 'Cont Size', 'Cont Type',
          'Cont No', 'SEAL', 'TIME SLOT', 'REMARKS', 'TD- STATUS']
master = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/EM master file.csv")
master = master.drop(columns=columns, axis=1)
master = master.iloc[:, 1:] #remove first row
master = master.reset_index().drop(columns='index')

from datetime import datetime
try:
    master['Coll Date'] = pd.to_datetime(master['Coll Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d') #format datetime
    master['DELIVERY'] = pd.to_datetime(master['DELIVERY'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
except:
    pass
master = master.iloc[1:]
master['EM SO# Ref'] = master['EM SO# Ref'].astype('int64')

In [12]:
temp_upload_cust = temp_upload_1['SAP Order No'].tolist()
temp_upload_cust = list(map(int, temp_upload_cust)) #convert list items to int

master = master[master['EM SO# Ref'].isin(temp_upload_cust)] #select rows based on list
#master = master[master['SAP Order No'].isin(temp_upload_cust)]
col_mas = ['Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT', 'LOADING PLANT', 'DELIVERY', 'EM SO# Ref'] #'EM SO# Ref'
master = master[col_mas]
master.rename(columns={'EM SO# Ref': 'SAP Order No'}, inplace=True)

master.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
master.index = np.arange(1, len(master)+1)

master['Cust Ref'].fillna(value=0, inplace=True)
master['Transport No'].fillna(value=0, inplace=True)
master['Cust Ref'] = master['Cust Ref'].astype('int64')
master['Transport No'] = master['Transport No'].astype('int64')
if len(master) == 0:
    print('\033[1m'+ '\033[94m'+ 'All download Transports are new')
master

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
1,1003981694,6108320268,17/1/2023,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"CHAMPION X 11 AYER MERBAU ROAD, JURONG ISLAND",17/1/2023,5944859
2,1003981634,6108320286,17/1/2023,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"CHAMPION X 11 AYER MERBAU ROAD, JURONG ISLAND",17/1/2023,5944862
3,1003982307,6108324475,31/1/2023,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,MEGACHEM MANUFATURING\n132 PIONEER ROAD,31/1/2023,5950321
4,1003981242,6108316731,16/1/2023,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXAL 8 ROP VL,BASF 20 BANYAN AVENUE JURONG ISLAND,16/1/2023,5954173
5,1003982696,6108328088,1/2/2023,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"CHAMPION X 11 AYER MERBAU ROAD, JURONG ISLAND",1/2/2023,5954952
6,1003982714,6108328087,1/2/2023,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"CHAMPION X 11 AYER MERBAU ROAD, JURONG ISLAND",1/2/2023,5955180
7,1003982739,6108328086,1/2/2023,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"CHAMPION X 11 AYER MERBAU ROAD, JURONG ISLAND",1/2/2023,5955532
8,1003982760,6108328089,2/2/2023,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"CHAMPION X 11 AYER MERBAU ROAD, JURONG ISLAND",2/2/2023,5955533
9,1003982715,6108328090,2/2/2023,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"CHAMPION X 11 AYER MERBAU ROAD, JURONG ISLAND",2/2/2023,5955534
10,1003982780,6108328092,3/2/2023,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"CHAMPION X 11 AYER MERBAU ROAD, JURONG ISLAND",3/2/2023,5955535


In [13]:
if len(master) != 0:
    master['Coll Date'] = pd.to_datetime(master['Coll Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d') #format datetime
    master['DELIVERY'] = pd.to_datetime(master['DELIVERY'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

In [14]:
df_sap = master.copy() #transports with changes
df_sap.rename(columns={'SAP Order No': 'SAP_Order_No'}, inplace=True)
sap = df_sap[df_sap.duplicated('SAP_Order_No', keep=False)].groupby('SAP_Order_No')['Coll Date'].apply(list).reset_index()
lst_sap = sap['SAP_Order_No'].tolist()
sap_1 = df_sap[df_sap["SAP_Order_No"].isin(lst_sap)]

if len(lst_sap) != 0:
    for i in range(len(sap)):
        print(sap.at[i, 'Coll Date'])
        print(lst_sap)
        #print(sap_1)
else:
    print('No transports changes')

No transports changes


C:\Users\john.tan\AppData\Local\Temp\ipykernel_14684\1141463110.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  sap = df_sap[df_sap.duplicated('SAP_Order_No', keep=False)].groupby('SAP_Order_No')['Coll Date'].apply(list).reset_index()


In [15]:
from datetime import datetime
for i in range(len(sap)):
    sap_order = sap.at[i, 'SAP_Order_No']
    coll_date = sap.at[i, 'Coll Date']
    coll_date_lst = [datetime.strptime(coll, "%Y-%m-%d").date() for coll in coll_date] #convert elem in list to date
    print(sap_order + ' Latest collection date: '+ max(coll_date_lst).strftime("%Y-%m-%d"))

In [16]:
#new transports works 16/01/2023!
uploaded_trans = df_trans['SAP Order No'].tolist()
master_trans = master['SAP Order No'].tolist()

uploaded_trans = np.unique(uploaded_trans).astype(float)
master_trans = np.unique(master_trans).astype(float)

print('New transport SAP Order Number ')
#np.setdiff1d(uploaded_trans, master_trans).astype(int)
uploaded_trans = [x for x in uploaded_trans if x not in master_trans]
uploaded_trans = list(map(int, uploaded_trans)) #convert to int
uploaded_trans = [ele for ele in uploaded_trans if ele > 0]
print('\033[1m'+ '\033[34m'+ str(uploaded_trans))

new_trans = [] #convert list of int to str
for x in uploaded_trans:
    str_x = "% s" % x
    new_trans.append(str_x)

df_newtrans = df_trans[df_trans['SAP Order No'].isin(new_trans)]
df_newtrans

New transport SAP Order Number 
[12734192, 12737576, 12737630, 12737631, 12737632, 12737633, 12737634, 12737635, 12739934, 12743420, 12743533, 12743534, 12744387, 12744388, 12744389, 4911877915, 5955539, 5955540]


,Cust Ref,Order Remarks,Transport No,Coll Date,DEPOT_MV2,PRODUCT,LOADING_PLANT_MV3,DELIVERY,SAP Order No,MV2
24,1003985251,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108348445,2023-02-09,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,EXXAL 8 ROP VL,BASF SOUTH EAST ASIA PTE LTD,2023-02-09,12734192,EXX003
25,1003985271,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108348448,2023-02-09,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-09,5955539,EXX003
26,1003985289,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108348449,2023-02-09,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-09,5955540,EXX003
27,1003985642,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108351515,2023-02-10,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-02-10,12737630,EXX004
28,1003985648,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108351514,2023-02-10,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-02-10,12737631,EXX004
29,1003985668,THE POLYOLEFIN - PULAU AYER MERBAUISOPAR L AP VL,6108351516,2023-02-10,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,ISOPAR L AP VL,THE POLYOLEFIN - PULAU AYER MERBAU,2023-02-10,12737576,EXX003
30,1003986443,SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN:...,6108356762,2023-02-06,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,EXXSOL HEPTANE AP VL,SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN:...,2023-02-06,12743420,EXX003
31,1003986608,SUMITOMO CHEMICAL ASIA PTE LTDEXXSOL HEXANE SP...,6108357583,2023-02-03,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,SUMITOMO CHEMICAL ASIA PTE LTD,2023-02-03,12743533,EXX004
34,1003987078,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)EXXSO...,6108361134,2023-02-13,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,EXXSOL HEPTANE AP VL,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),2023-02-13,12744389,EXX003
35,1003987107,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)EXXSO...,6108361133,2023-02-08,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,EXXSOL HEPTANE AP VL,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),2023-02-08,12744388,EXX003


In [17]:
#comp (iterate loading plant) with color, write to excel works 20/1/2023!
#import numpy as np

def highlight_diff(data, color='lightgreen'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('First', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

def repeatRows(d, n):
    return pd.concat([d]*n)

temp_upload_6 = temp_upload_1.copy()
master_6 = master.copy()
tp = temp_upload_6.groupby('LOADING PLANT')   
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df
ms = master_6.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

for i in range(min(len(lst_tp), len(lst_ms))):
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    if n!= 0:

        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])
            
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
            
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        #print(df_comp)
        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+ str(i) + datetime.now().strftime("%Y%m%d_%H%M"), engine='openpyxl')
        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/Comp_color/comp.xlsx", mode="a", engine="openpyxl") as writer1:
            df_comp.to_excel(writer1, sheet_name='comp_'+ str(i) + datetime.now().strftime("%Y%m%d_%H%M"), engine='openpyxl')

C:\Users\john.tan\AppData\Local\Temp\ipykernel_14684\2937698046.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_14684\2937698046.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_14684\2937698046.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_14684\2937698046.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be ke

In [ ]:
class MyList(list):
    def __contains__(self, typ):
        return any(isinstance(val, typ) for val in self)

x = MyList(uploaded_trans)
print (int in x)

In [48]:
#optional
def repeatRows(d, n):
    return pd.concat([d]*n)

n = abs(len(master) - len(temp_upload_1)) #ensure both df same len
if n!= 0:
    if len(master) > len(temp_upload_1):
        temp_upload_1 = pd.concat([temp_upload_1, repeatRows(temp_upload_1[-1:], n)])
        temp_upload_1.index = np.arange(1, len(temp_upload_1)+1)
        #print(len(temp_upload_1))
    else:
        master = pd.concat([master, repeatRows(master[-1:], n)])
        master.index = np.arange(1, len(master)+1)
        #print(len(master))
else:
    print('No existing transports to compare')

In [49]:
df_all = pd.concat([temp_upload_1, master], axis='columns', keys=['First', 'Second'])
df_comp = df_all.swaplevel(axis='columns')[temp_upload_1.columns[1:]]

def highlight_diff(data, color='lightgreen'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('First', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

master = master.dropna(subset=['Transport No','Coll Date'], how='all')
temp_upload_1 = temp_upload_1.dropna(subset=['Transport No','Coll Date'], how='all')

df_comp = df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)
#df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)
df_comp

C:\Users\john.tan\AppData\Local\Temp\ipykernel_29352\2874637043.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)


In [50]:
master.compare(temp_upload_1, keep_shape=True, keep_equal=True)

Cust Ref             Transport No               Coll Date              \
          self       other         self       other        self       other   
1   1003981694  1003981694   6108320268  6108320268  2023-01-17  17/01/2023   
2   1003981634  1003981634   6108320286  6108320286  2023-01-17  17/01/2023   
3   1003979986  1003979986   6108310255  6108310255  2023-01-26  26/01/2023   
4   1003980015  1003980015   6108310254  6108310254  2023-01-26  26/01/2023   
5   1003980638  1003980638   6108312664  6108312664  2023-01-16  16/01/2023   
6   1003982307  1003982307   6108324475  6108324475  2023-01-31  31/01/2023   
7   1003981242  1003981242   6108316731  6108316731  2023-01-16  16/01/2023   
8   1003982696  1003982696   6108328088  6108328088  2023-02-01  01/02/2023   
9   1003982714  1003982714   6108328087  6108328087  2023-02-01  01/02/2023   
10  1003982739  1003982739   6108328086  6108328086  2023-02-01  01/02/2023   
11  1003982760  1003982760   6108328089  6108328089  2023-02-02  02/02/2023   
12  1003982715  1003982715   6108328090  6108328090  2023-02-02  02/02/2023   
13  1003982780  1003982780   6108328092  6108328092  2023-02-03  03/02/2023   
14  1003982733  1003982733   6108328091  6108328091  2023-02-03  03/02/2023   
15  1003983928  1003983928   6108337466  6108337466  2023-02-06  06/02/2023   
16  1003983891  1003983891   6108337465  6108337465  2023-02-06  06/02/2023   
17  1003980836  1003985271   6108314616  6108348448  2023-01-27  09/02/2023   
18  1003980889  1003985289   6108314617  6108348449  2023-01-27  09/02/2023   
19  1003982292  1003980836   6108324472  6108314616  2023-01-31  27/01/2023   
20  1003982230  1003980889   6108324473  6108314617  2023-01-31  27/01/2023   
21  1003979353  1003982292   6108305780  6108324472  2023-01-25  31/01/2023   
22  1003980908  1003982230   6108314618  6108324473  2023-01-27  31/01/2023   
23  1003982293  1003979353   6108324476  6108305780  2023-01-31  25/01/2023   
24  1003982065  1003980908   6108322714  6108314618  2023-01-30  27/01/2023   
25  1003980890  1003982293   6108314619  6108324476  2023-01-27  31/01/2023   
26  1003982074  1003982065   6108322715  6108322714  2023-01-30  30/01/2023   
27  1003981261  1003980890   6108317029  6108314619  2023-01-20  27/01/2023   
28           0  1003982074            0  6108322715  2023-02-13  30/01/2023   
29  1003981271  1003981261   6108317031  6108317029  2023-01-27  20/01/2023   
30  1003982231  1003985668   6108324474  6108331208  2023-01-31  13/02/2023   
31  1003984405  1003981271   6108341829  6108317031  2023-02-07  27/01/2023   
32  1003983117  1003982231   6108330748  6108324474  2023-01-31  31/01/2023   
33  1003983009  1003984405   6108330411  6108341829  2023-02-01  07/02/2023   
34  1003983150  1003985251   6108331206  6108348445  2023-02-03  09/02/2023   
35  1003983151  1003983117   6108331207  6108330748  2023-02-03  31/01/2023   
36  1003984487  1003983009   6108341827  6108330411  2023-02-07  01/02/2023   
37  1003984483  1003985668   6108341828  6108351516  2023-02-07  10/02/2023   
38  1003984119  1003983150   6108339662  6108331206  2023-01-31  03/02/2023   
39  1003984462  1003983151   6108341830  6108331207  2023-02-07  03/02/2023   
40  1003983178  1003984487   6108331205  6108341827  2023-01-20  07/02/2023   
41  1003983178  1003984483   6108331205  6108341828  2023-01-20  07/02/2023   
42  1003983178  1003985642   6108331205  6108351515  2023-01-20  10/02/2023   
43  1003983178  1003985648   6108331205  6108351514  2023-01-20  10/02/2023   
44  1003983178  1003984119   6108331205  6108339662  2023-01-20  31/01/2023   
45  1003983178  1003984462   6108331205  6108341830  2023-01-20  07/02/2023   
46  1003983178  1003983178   6108331205  6108331205  2023-01-20  20/01/2023   

                                         DEPOT  \
                                          self   
1   Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
2   Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
3   Vopak (Pe

In [18]:
#df_trans = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/temp_upload.csv") #Textract
trans_1 = df_trans[df_trans['DEPOT_MV2'].str.contains('VOPAK', na=False)]
#trans_1 = df_trans[~df_trans['DEPOT_MV2'].str.contains('VOPAK', na=False)]
vopak = trans_1.copy()
#vopak = vopak.iloc[:, 1:] #drop first col
vopak.reset_index(drop=True, inplace=True)

#vopak.insert(loc=0, column='Transporter Name', value='SH COGENT LOGISTICS PTE LTD')
#vopak.insert(loc=0, column='Customer Name', value='ExxonMobil Chemical Asia Pacific')

vopak = vopak.drop(columns=['Order Remarks', 'Transport No', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 'MV2'])
#vopak = vopak.drop(columns=['Order Remarks', 'Transport No', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 'MV2'])
#vopak.rename(columns={'Cust Ref': 'Customer Reference Number', 'Coll Date': 'Schedule Date', 'PRODUCT': 'Product name'}, inplace=True)
vopak.rename(columns={'Coll Date': 'Schedule Date', 'PRODUCT': 'Product name'}, inplace=True)

In [19]:
vopak

,Cust Ref,Schedule Date,Product name,SAP Order No
0,1003982293,2023-01-31,EXXSOL HEXANE SP AP VL,12726160
1,1003982307,2023-01-31,SOLVESSO 150 AP VL,5950321
2,1003982696,2023-02-01,SOLVESSO 150 AP VL,5954952
3,1003982714,2023-02-01,SOLVESSO 150 AP VL,5955180
4,1003982715,2023-02-02,SOLVESSO 150 AP VL,5955534
5,1003982733,2023-02-03,SOLVESSO 150 AP VL,5955536
6,1003982739,2023-02-01,SOLVESSO 150 AP VL,5955532
7,1003982760,2023-02-02,SOLVESSO 150 AP VL,5955533
8,1003982780,2023-02-03,SOLVESSO 150 AP VL,5955535
9,1003981242,2023-01-16,EXXAL 8 ROP VL,5954173


In [20]:
#Vopak email within 10 working days
from datetime import datetime, timedelta 
#acc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Accepted_transports.csv")
vopak['Schedule Date'] = pd.to_datetime(vopak['Schedule Date'])
curr = datetime.now()
end = curr + timedelta(days=14)
start = curr - timedelta(days=1)

vopak_1 = vopak[vopak['Schedule Date'].between(start, end)]
lst_vopak = vopak_1['Cust Ref'].tolist()
if len(vopak_1) != 0:
    print(vopak_1)
else:
    print('No Vopak transports in next 10 business days')

send_vopak = vopak_1.copy() #make copy of vopak_1 df

      Cust Ref Schedule Date            Product name SAP Order No
0   1003982293    2023-01-31  EXXSOL HEXANE SP AP VL     12726160
1   1003982307    2023-01-31      SOLVESSO 150 AP VL      5950321
2   1003982696    2023-02-01      SOLVESSO 150 AP VL      5954952
3   1003982714    2023-02-01      SOLVESSO 150 AP VL      5955180
4   1003982715    2023-02-02      SOLVESSO 150 AP VL      5955534
5   1003982733    2023-02-03      SOLVESSO 150 AP VL      5955536
6   1003982739    2023-02-01      SOLVESSO 150 AP VL      5955532
7   1003982760    2023-02-02      SOLVESSO 150 AP VL      5955533
8   1003982780    2023-02-03      SOLVESSO 150 AP VL      5955535
10  1003983009    2023-02-01      ACTREL 3356L AP VL     12737491
11  1003983117    2023-01-31      ACTREL 3356L AP VL     12737490
13  1003983891    2023-02-06      SOLVESSO 150 AP VL      5955538
14  1003983928    2023-02-06      SOLVESSO 150 AP VL      5955537
15  1003984119    2023-01-31          ISOPAR L AP VL     12739889
16  100398

In [21]:
def copyCertainFiles(source_folder, dest_folder, string_to_match, file_type='.pdf'):
    # Check all files in source_folder
    for filename in os.listdir(source_folder):
        # Move the file if the filename contains the string to match
        if file_type == file_type:
            if string_to_match in filename:
                shutil.move(os.path.join(source_folder, filename), dest_folder)

        # Check if the keyword and the file type both match
        elif isinstance(file_type, str):
            if string_to_match in filename and file_type in filename:
                shutil.move(os.path.join(source_folder, filename), dest_folder)

source_folder = r'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/'
dest_folder = r'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Send_Vopak/'
file_type = '.pdf'

for cust_ref in lst_vopak:
    string_to_match = cust_ref
    copyCertainFiles(source_folder, dest_folder, string_to_match, file_type)

#close all pdf files
#https://stackoverflow.com/questions/61620485/python-if-filename-in-specified-path-contains-string-then-move-to-folder
#move pdf within 10 working days

In [22]:
#outlook to VOPAK
from importlib import reload #send outlook multiple attachments to multiple recipients 28/11/2022 great!!
import win32com.client as win32
import pythoncom
import sys
import os
import pandas as pd
import pdfplumber
from datetime import datetime

reload(sys)
pythoncom.CoInitialize()
outlook = win32.Dispatch('outlook.application')

def sendmail(receiver, attachment, subject):
    receiver = receiver
    attachment = attachment
    sub = subject
    body = """Dear VOPAK
    <br><br>(Test email) Kindly release the booking in the system for the orders.
    <br>Customer Name: ExxonMobil Chemical Asia Pacific
    <br>Transporter Name: SH Cogent Logistics Pte Ltd<br><br>""" + vopak_1.to_html() +'<br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')

    for i in range(len(attachment)):
        mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()
    
path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/" #copy Vopak pdf to Vopak folder (outlook)
dst = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/Chawan_Vopak_email/Send_Vopak/"
for file in glob.glob(path_dir+"*.pdf"):
    #file = str(file)
    path_from = file
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    try:
        ori = regexp.search(text).group(1).split('SG-',1)[0].split()[1]
        if ori.lower() in ['VOPAK', 'vopak']:
            shutil.copy(file, dst) #if contains 'vopak', copy/paste file
            pdf.close()
    except:
        pass
    
path = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/Chawan_Vopak_email/Send_Vopak/" # Folder of attachments
addr = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Address.csv") # Email address

for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "VOPAK - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    sendmail(addr["add"][j], dirlist, subjects) # Send emails to multiple recipients with corresponding attachments

In [23]:
def newtrans_mail(receiver, attachment, subject):
    receiver = receiver
    attachment = attachment
    sub = subject
    body = """Dear User
    <br>(Test email) New transport orders from Transporeon.
    <br><br>""" + df_newtrans.to_html() +"<br><br>SAP Order Number<br>" + str(uploaded_trans) + '<br><br>Download latest pdf from Transporeon<br>' + df_trans.to_html() +'<br><br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')

    for i in range(len(attachment)):
        mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()

##outlook to user download latest pdf Transporeon, comp.xlsx
path = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/" # Folder of comp.xlsx
addr = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Address.csv") # Email address
    
for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "New - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    newtrans_mail(addr["add"][j], dirlist, subjects) # Send emails to multiple recipients with corresponding attachments

John
Charles
Chia Mean


In [24]:
regexp = re.compile("Destination(.*)$", re.MULTILINE) #TRANSPOREON upload to TMS works 20/1/2023!
regexp_poly = re.compile("PENJURU ROAD(.*)$", re.MULTILINE)

tms_lst = []
tms = pd.DataFrame(columns=['Delivery Type No', 'MV2', 'MV3'])

lst_cust = df_trans['LOADING_PLANT_MV3'].tolist()

#conditions=[cust=='EXXONMOBIL CHEMICAL COMPANY', cust=='MITSUI ELASTOMERS SINGAPORE PTE LTD', cust=='SUMITOMO CHEMICAL PTE LTD',
            #cust=='A.I.M. CHEMICAL INDUSTRIES PTE LTD', cust=='CLP INTERNATIONAL PTE LTD', cust=='CHAMPIONX SG SERVICE PTE LTD',
            #cust=='SINGAPORE ADHESION PLANT', cust=='BASF SOUTH EAST ASIA PTE LTD', cust=='ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD',
            #cust=='MEGACHEM MANUFACTURING PTE LTD', cust=='MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)', cust=='SINGAPORE BUTYL PLANT',
            #cust=='BERTSCHI', cust=='BAKER HUGHES', cust=='HP SINGAPORE', cust=='TOTALENERGIES', cust=='SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG']

#values=['EXX006', 'MITSUI', 'SUMITOMO CHEMICAL', 'EXX005', 'CLP001', 'CX', 'SAP', 'BASF', 'AKSS', 'MEGA', 'MSD', 'SBP', 'BERTSCHI', '627454', 'EXX002', 'TOTAL', 'SEIKA']
#values_1 = [1, 4, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1]

for cust in lst_cust:
    cust = cust.strip()
    conditions=[cust=='EXXONMOBIL CHEMICAL COMPANY', cust=='MITSUI ELASTOMERS SINGAPORE PTE LTD', cust=='SUMITOMO CHEMICAL PTE LTD',
            cust=='A.I.M. CHEMICAL INDUSTRIES PTE LTD', cust=='CLP INTERNATIONAL PTE LTD', cust=='CHAMPIONX SG SERVICE PTE LTD',
            cust=='SINGAPORE ADHESION PLANT', cust=='BASF SOUTH EAST ASIA PTE LTD', cust=='ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD',
            cust=='MEGACHEM MANUFACTURING PTE LTD', cust=='MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)', cust=='SINGAPORE BUTYL PLANT',
            cust=='BERTSCHI', cust=='BAKER HUGHES', cust=='HP SINGAPORE', cust=='TOTALENERGIES', cust=='SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG',
            cust=='THE POLYOLEFIN - PULAU AYER MERBAU']

    values=['EXX006', 'MITSUI', 'SUMITOMO CHEMICAL', 'EXX005', 'CLP001', 'CX', 'SAP', 'BASF', 'AKSS', 'MEGA', 'MSD', 'SBP', 'BERTSCHI', '627454', 'EXX002', 'TOTAL', 'SEIKA', 'TPCPAM2']
    values_1 = [1, 4, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4]
    
    MV2 = np.select(conditions, values)
    deli_type = np.select(conditions, values_1)
    print('customer is: '+ str(cust) + ' MV2: '+ str(MV2) + ' Delivery_Type: '+str(deli_type))
    print('------------------')

    #if str(cust).strip() == 'THE POLYOLEFIN - PULAU AYER MERBAU':
            #dest = re.search(regexp_poly, text).group(0)
            #if str(dest) =='PENJURU ROAD 59 STREET 2 GATE NO 5 401' or str(dest) == 'PENJURU ROAD 59 STREET 2 GATE NO 23 401':
                #MV2 = 'TPCPAM2'
                #deli_type = 4
                #print('customer is '+ str(cust) + ' '+ str(MV2) + ' ' + str(deli_type))
            #else:
                #MV2 = 'TPCPAM1'
                #deli_type = 4
                #print('customer is '+ str(dest) + str(MV2) + ' '+ str(deli_type))
    tms_lst.extend([[str(deli_type), str(MV2), cust]])
tms = pd.DataFrame(tms_lst, columns=['Delivery_Type', 'MV2', 'LOADING_PLANT_MV3']) #list of lists to df

customer is: MITSUI ELASTOMERS SINGAPORE PTE LTD MV2: MITSUI Delivery_Type: 4
------------------
customer is: MITSUI ELASTOMERS SINGAPORE PTE LTD MV2: MITSUI Delivery_Type: 4
------------------
customer is: THE POLYOLEFIN - PULAU AYER MERBAU MV2: TPCPAM2 Delivery_Type: 4
------------------
customer is: MEGACHEM MANUFACTURING PTE LTD MV2: MEGA Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: CHAMPIONX SG SERVICE PTE LTD MV2: CX Delivery_Type: 1
------------------
customer is: BASF

In [25]:
tms

,Delivery_Type,MV2,LOADING_PLANT_MV3
0,4,MITSUI,MITSUI ELASTOMERS SINGAPORE PTE LTD
1,4,MITSUI,MITSUI ELASTOMERS SINGAPORE PTE LTD
2,4,TPCPAM2,THE POLYOLEFIN - PULAU AYER MERBAU
3,1,MEGA,MEGACHEM MANUFACTURING PTE LTD
4,1,CX,CHAMPIONX SG SERVICE PTE LTD
5,1,CX,CHAMPIONX SG SERVICE PTE LTD
6,1,CX,CHAMPIONX SG SERVICE PTE LTD
7,1,CX,CHAMPIONX SG SERVICE PTE LTD
8,1,CX,CHAMPIONX SG SERVICE PTE LTD
9,1,CX,CHAMPIONX SG SERVICE PTE LTD


In [413]:
from PyPDF2 import PdfReader, PdfFileReader #TMS file development
import re
import glob, os
import pdfplumber
import numpy as np
import pandas as pd

regexp = re.compile("Destination(.*)$", re.MULTILINE)
regexp_poly = re.compile("PENJURU ROAD(.*)$", re.MULTILINE)

tms_lst = []
tms = pd.DataFrame(columns=['Delivery Type No', 'MV2', 'MV3'])
path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Test/Test_2/"
for file in glob.glob(path_dir + "*.pdf"):

    pdf = pdfplumber.open(file)
    pdf_pg_count = len(pdf.pages)
    page = pdf.pages[0]
    #p = page[i]
    text = page.extract_text()
    #print(text)
    
    cust = re.search(regexp, text).group(1).split('SG-')[0] #MV3
    cust = cust.strip()
    conditions=[cust=='EXXONMOBIL', cust=='MITSUI', cust=='SUMITOMO', cust=='A.I.M. CHEMICAL', cust=='CLP INTERNATIONAL', cust=='CHAMPIONX SG', cust=='SINGAPORE', cust=='BASF SOUTH EAST', cust=='ASAHI KASEI', cust=='MEGACHEM', cust=='MSD', cust=='SINGAPORE BUTYL', cust=='BERTSCHI', cust=='BAKER HUGHES', cust=='HP SINGAPORE', cust=='TOTALENERGIES', cust=='SUMITOMO SEIKA-17']
    values=['EXX006', 'MITSUI', 'SUMITOMO CHEMICAL', 'EXX005', 'CLP001', 'CX', 'SAP', 'BASF', 'AKSS', 'MEGA', 'MSD', 'SBP', 'BERTSCHI', '627454', 'EXX002', 'TOTAL', 'SEIKA']
    values_1 = [1, 4, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1]
    
    MV2 = np.select(conditions, values)
    deli_type = np.select(conditions, values_1)
    print('customer is: '+ str(cust) + ' '+ str(MV2) + ' '+str(deli_type))
    print('------------------')
    if str(cust).strip() == 'THE POLYOLEFIN -':
            
            dest = re.search(regexp_poly, text).group(0)
            if str(dest) =='PENJURU ROAD 59 STREET 2 GATE NO 5 401' or str(dest) == 'PENJURU ROAD 59 STREET 2 GATE NO 23 401':
                MV2 = 'TPCPAM2'
                print('customer is '+ str(dest) + ' '+ str(MV2))
            else:
                MV2 = 'TPCPAM1'
                deli_type = 4
                print('customer is '+ str(dest) + str(MV2) + ' '+ str(deli_type))
            pdf.close()
    tms_lst.extend([[str(deli_type), str(MV2), cust]])
    pdf.close()
#tms_lst
tms = pd.DataFrame(tms_lst, columns=['Delivery_Type', 'MV2', 'Customer_MV3']) #list of lists to df

customer is: MITSUI MITSUI 4
------------------
customer is: MITSUI MITSUI 4
------------------
customer is: EXXONMOBIL EXX006 1
------------------
customer is: ASAHI KASEI AKSS 4
------------------
customer is: MEGACHEM MEGA 1
------------------
customer is: MEGACHEM MEGA 1
------------------
customer is: CHAMPIONX SG CX 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: THE POLYOLEFIN - 0 0
------------------
customer is PENJURU ROAD 59 STREET 2 GATE NO 23 401 TPCPAM2
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: ASAHI KASEI AKSS 4
------------------
customer is: TOTALENERGIES TOTAL 1
------------------
customer is: SUMITOMO SEIKA-17 SEIKA 1
--------------

In [26]:
tms_upload = pd.concat([tms, df_trans], axis=1)
tms_upload #template for TMS upload

,Delivery_Type,MV2,LOADING_PLANT_MV3,Cust Ref,Order Remarks,Transport No,Coll Date,DEPOT_MV2,PRODUCT,LOADING_PLANT_MV3,DELIVERY,SAP Order No,MV2
0,4,MITSUI,MITSUI ELASTOMERS SINGAPORE PTE LTD,1003980889,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108314617,2023-01-27,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-01-27,12720815,EXX004
1,4,MITSUI,MITSUI ELASTOMERS SINGAPORE PTE LTD,1003982292,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108324472,2023-01-31,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-01-31,12720816,EXX004
2,4,TPCPAM2,THE POLYOLEFIN - PULAU AYER MERBAU,1003982293,THE POLYOLEFIN - PULAU AYER MERBAUEXXSOL HEXAN...,6108324476,2023-01-31,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,EXXSOL HEXANE SP AP VL,THE POLYOLEFIN - PULAU AYER MERBAU,2023-01-31,12726160,EXX003
3,1,MEGA,MEGACHEM MANUFACTURING PTE LTD,1003982307,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108324475,2023-01-31,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,2023-01-31,5950321,EXX003
4,1,CX,CHAMPIONX SG SERVICE PTE LTD,1003982696,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108328088,2023-02-01,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-01,5954952,EXX003
5,1,CX,CHAMPIONX SG SERVICE PTE LTD,1003982714,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108328087,2023-02-01,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-01,5955180,EXX003
6,1,CX,CHAMPIONX SG SERVICE PTE LTD,1003982715,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108328090,2023-02-02,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-02,5955534,EXX003
7,1,CX,CHAMPIONX SG SERVICE PTE LTD,1003982733,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108328091,2023-02-03,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-03,5955536,EXX003
8,1,CX,CHAMPIONX SG SERVICE PTE LTD,1003982739,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108328086,2023-02-01,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-01,5955532,EXX003
9,1,CX,CHAMPIONX SG SERVICE PTE LTD,1003982760,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108328089,2023-02-02,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2023-02-02,5955533,EXX003


In [27]:
import xlsxwriter #write df_comp

writer = pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/comp.xlsx", engine='xlsxwriter')
sheetComp = 'Comp_' + datetime.now().strftime("%Y%m%d_%H%M")
df_comp.to_excel(writer, sheet_name=sheetComp) #write to excel with color
worksheet1 = writer.sheets[sheetComp]
worksheet1.set_tab_color('blue')
writer.save()
writer.close()

C:\Users\john.tan\AppData\Local\Temp\ipykernel_14684\1251628687.py:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\users\john.tan\appdata\local\programs\python\python39\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [28]:
def dataframe_difference(df1, df2, which):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(
        df2,
        indicator=True,
        how='outer'
    )
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    #diff_df.to_csv('data/diff.csv')
    return diff_df

temp_upload_1['Cust Ref'] = temp_upload_1['Cust Ref'].astype('int64')
temp_upload_1['Transport No'] = temp_upload_1['Transport No'].astype('int64')
temp_upload_1['SAP Order No'] = temp_upload_1['SAP Order No'].astype('int64')
new_upload = dataframe_difference(temp_upload_1, master, 'left_only')
new_upload = new_upload.rename(columns={'_merge': 'New_Download'})
new_upload
#temp_upload_2 = temp_upload_1.groupby('LOADING PLANT').apply(lambda x: x.sort_values('SAP Order No'))

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No,New_Download
0,1003981694,6108320268,17/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,17/01/2023,5944859,left_only
1,1003981634,6108320286,17/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,17/01/2023,5944862,left_only
2,1003982307,6108324475,31/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,31/01/2023,5950321,left_only
3,1003981242,6108316731,16/01/2023,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,EXXAL 8 ROP VL,BASF SOUTH EAST ASIA PTE LTD,16/01/2023,5954173,left_only
4,1003982696,6108328088,01/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,01/02/2023,5954952,left_only
5,1003982714,6108328087,01/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,01/02/2023,5955180,left_only
6,1003982739,6108328086,01/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,01/02/2023,5955532,left_only
7,1003982760,6108328089,02/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,02/02/2023,5955533,left_only
8,1003982715,6108328090,02/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,02/02/2023,5955534,left_only
9,1003982780,6108328092,03/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,03/02/2023,5955535,left_only


In [465]:
#optional
tp = temp_upload_1.groupby('LOADING PLANT')    
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df

ms = master.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

def repeatRows(d, n):
    return pd.concat([d]*n)

for i in range(min(len(lst_tp), len(lst_ms))):
        
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    if n!= 0:
        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
            #print(len(lst_tp[i]))
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])
            #print(len(lst_ms[i]))
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        print(lst_ms[i].compare(lst_tp[i], keep_shape=True, keep_equal=True)) #compare side by side
        
#https://stackoverflow.com/questions/23691133/split-pandas-dataframe-based-on-groupby

     Cust Ref             Transport No              Coll Date              \
         self       other         self       other       self       other   
1  1003974560  1003974560   6108272471  6108272471  13/1/2023         NaN   
2  1003978428  1003974560   6108298482  6108272471  18/1/2023  01/11/2023   
3  1003976739  1003978428   6108288398  6108298482  19/1/2023  13/01/2023   
4  1003976739  1003979353   6108288398  6108305780  19/1/2023         NaN   

                                        DEPOT  \
                                         self   
1  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
2  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
3  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
4  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   

                                                                     PRODUCT  \
                                               other                    self   
1  T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...  EXXSOL HEXANE SP AP VL   
2 

In [474]:
#alternative comparison optional
tp6 = temp_upload_1.copy()
ms6 = master.copy()
tp6.drop(columns={'Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT'}, inplace=True)
ms6.drop(columns={'Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT'}, inplace=True)

diffs = set(ms6.iloc[:, :3].apply(tuple, 1)) \
- set(tp6.apply(tuple, 1))
lst_diffs = list(diffs)

50


In [486]:
#optional
lst_tp[11].index = np.arange(1, len(lst_tp[11])+1)
lst_ms[11].index = np.arange(1, len(lst_ms[11])+1)
lst_ms[11].compare(lst_tp[11], keep_shape=True, keep_equal=True)

Cust Ref             Transport No              Coll Date              \
         self       other         self       other       self       other   
1  1003977817  1003976424   6108295225  6108286474  11/1/2023  16/01/2023   
2  1003977817  1003977817   6108295225  6108295225  11/1/2023  18/01/2023   

                                        DEPOT  \
                                         self   
1  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
2  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   

                                                           PRODUCT  \
                                               other          self   
1  T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...  SPECTRASYN 4   
2  T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...  SPECTRASYN 4   

                                                   LOADING PLANT  \
        other                                               self   
1  MCP2542 VL  TOTALENERGIES MARKETING ASIA-PACIFIC MIDDLE EA...   
2  MCP2542 VL  TOTALENERGIES MARKETING ASIA-PACIFIC MIDDLE EA...   

                                                       DELIVERY              \
                                               other       self       other   
1  TOTALENERGIES MARKETING ASIA- PACIFIC MIDDLE E...  11/1/2023  10/01/2023   
2  TOTALENERGIES MARKETING ASIA- PACIFIC MIDDLE E...  11/1/2023  11/01/2023   

  SAP Order No            
          self     other  
1     12728630  12727043  
2     12728630  12728630

In [119]:
# (development) comp in color to excel works 17/1/2023!
temp_upload_6 = temp_upload_1.copy()
master_6 = master.copy()
tp = temp_upload_6.groupby('LOADING PLANT')   
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df
ms = master_6.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

for i in range(min(len(lst_tp), len(lst_ms))):
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    
    if n!= 0:

        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])           
          
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)      
           
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        #df_comp
        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/Comp_color/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+str(i)+'_'+datetime.now().strftime("%Y-%m-%d"), engine='openpyxl')
               

    if n ==0:    
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
                
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+str(i)+'_'+datetime.now().strftime("%Y-%m-%d"), engine='openpyxl')

C:\Users\john.tan\AppData\Local\Temp\ipykernel_29352\571956376.py:53: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_29352\571956376.py:53: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_29352\571956376.py:32: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
C:\Users\john.tan\AppData\Local\Temp\ipykernel_29352\571956376.py:32: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keywor

In [29]:
#compare Upcoming with master 18/01/2023
upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Upcoming.csv", usecols=['Order No'])
upc.rename(columns={'Order No': 'SAP Order No'}, inplace=True) #'Deliv Date': 'DELIVERY'
#upc['DELIVERY'] = pd.to_datetime(upc['DELIVERY'], format='%Y%m%d').dt.strftime('%d/%m/%Y')
upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
lst_upc = upc['SAP Order No'].tolist()

new_upc = [] #convert list of int to str
for x in lst_upc:
    str_x = "% s" % x
    new_upc.append(str_x)

master_upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/EM master file.csv", usecols=['EM SO# Ref'])
master_upc.rename(columns={'EM SO# Ref': 'SAP Order No'}, inplace=True)
master_upc = master_upc[master_upc['SAP Order No'].isin(new_upc)]
master_upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
master_upc.index = np.arange(1, len(master_upc)+1)

sap_master_upc = master_upc['SAP Order No'].tolist()

master_upc_1 = master_upc.copy()
master_upc_1['SAP Order No'] = master_upc_1['SAP Order No'].astype(np.int64) #col to int
upOnly = dataframe_difference(upc, master_upc_1, 'left_only')

upOnly.rename(columns={"_merge": "New_Transport"}, inplace=True)
upOnly['New_Transport'] = upOnly['New_Transport'].replace('left_only', 'new upcoming')
upOnly.index = np.arange(1, len(upOnly)+1)
print('\033[1m'+ '\033[34m'+ 'Transports from Upcoming not in MasterFile') #'\033[94m' +
upOnly

Transports from Upcoming not in MasterFile


,SAP Order No,New_Transport
1,5934621,new upcoming
2,5934622,new upcoming
3,5942922,new upcoming
4,5944856,new upcoming
5,5944857,new upcoming
6,5944859,new upcoming
7,5944862,new upcoming
8,5944864,new upcoming
9,5944867,new upcoming
10,5944868,new upcoming


In [35]:
#compare Upcoming with master (development)
upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Upcoming.csv", usecols=['Order No', 'Deliv Date', 'Customer Name'])
upc.rename(columns={'Order No': 'SAP Order No', 'Customer Name': 'LOADING', 'Deliv Date': 'DELIVERY'}, inplace=True)
upc['DELIVERY'] = pd.to_datetime(upc['DELIVERY'], format='%Y%m%d').dt.strftime('%d/%m/%Y')
upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
lst_upc = upc['SAP Order No'].tolist()

new_upc = [] #convert list of int to str
for x in lst_upc:
    str_x = "% s" % x
    new_upc.append(str_x)

master_upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/EM master file.csv", usecols=['EM SO# Ref', 'DELIVERY', 'LOADING PLANT'])
master_upc.rename(columns={'EM SO# Ref': 'SAP Order No', 'LOADING PLANT': 'LOADING'}, inplace=True)
master_upc = master_upc[master_upc['SAP Order No'].isin(new_upc)]
master_upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')

sap_master_upc = master_upc['SAP Order No'].tolist()
#print(sap_master_upc) #list of str
#print(lst_upc) #list of int

upc['SAP Order No'] = upc['SAP Order No'].astype(str) #convert col to str
upc_notin_master = upc[~upc['SAP Order No'].isin(sap_master_upc)] #select Upcoming not in Master
upc_notin_master.index = np.arange(1, len(upc_notin_master)+1)
#print('Upcoming transports not in Master file')
#print(upc_notin_master)

idx1 = set(upc.set_index(['SAP Order No', 'LOADING', 'DELIVERY']).index)
idx2 = set(upc_notin_master.set_index(['SAP Order No', 'LOADING', 'DELIVERY']).index)
upc_in_master = pd.DataFrame(list(idx1 - idx2), columns=upc.columns) #select Upcoming in Master

upc_in_master.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
upc_in_master.index = np.arange(1, len(upc_in_master)+1)
print('Upcoming transports in Master file')
print(upc_in_master)

master_upc.index = np.arange(1, len(master_upc)+1)
print('Master file')
print(master_upc)

#diffs = set(master_upc.iloc[:, :3].apply(tuple, 1)) \
                       #-  set(upc.apply(tuple, 1))
#print('Difference in master ')
#print(diffs)
diffs_2 = set(upc.iloc[:, :3].apply(tuple, 1)) \
                       -  set(master_upc.apply(tuple, 1))
#print('New entry in Upcoming transports')
DF_diffs_2 = pd.DataFrame(data=list(diffs_2), columns=['SAP Order No', 'LOADING', 'DELIVERY'])
#DF_diffs_2
#https://stackoverflow.com/questions/46635439/compare-rows-of-two-data-frames-with-unequal-lengths
#https://stackoverflow.com/questions/37313691/how-to-remove-a-pandas-dataframe-from-another-dataframe

Upcoming transports in Master file
Empty DataFrame
Columns: [SAP Order No, LOADING, DELIVERY]
Index: []
Master file
Empty DataFrame
Columns: [SAP Order No, LOADING, DELIVERY]
Index: []


In [35]:
from importlib import reload #send outlook multiple recipients 18/1/2023 great!!
import win32com.client as win32
import pythoncom
import sys
import os
import pandas as pd
import pdfplumber
from datetime import datetime

reload(sys)
pythoncom.CoInitialize()
outlook = win32.Dispatch('outlook.application')

def upctrans_mail(receiver, attachment, subject):
    receiver = receiver
    #attachment = attachment
    sub = subject
    body = """Dear User
    <br>(Test email) EM MasterFile.
    <br><br>""" + master_upc.to_html() +"<br><br>Upcoming Transports not in MasterFile<br>" + upOnly.to_html()+'<br><br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')

    #for i in range(len(attachment)):
        #mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()

##outlook to user download latest pdf Transporeon, comp.xlsx
path = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Test/Upcoming/" # Folder of comp.xlsx
addr = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Address.csv") # Email address
    
for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "Upcoming - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    upctrans_mail(addr["add"][j], dirlist, subjects) # Send emails to multiple recipients with corresponding attachments

John
Charles
Chia Mean


In [30]:
temp_upload_diff = temp_upload_1[['SAP Order No', 'LOADING PLANT', 'DELIVERY']]
temp_upload_diff.rename(columns={'LOADING PLANT': 'LOADING'}, inplace=True)
diffs_3 = set(temp_upload_diff.iloc[:, :3].apply(tuple, 1)) \
                       -  set(master.apply(tuple, 1))
print('New entry in Uploaded transports')
DF_diffs_3 = pd.DataFrame(data=list(diffs_3), columns=['SAP Order No', 'LOADING', 'DELIVERY'])
DF_diffs_3 #Upcoming not in master

New entry in Uploaded transports


C:\Users\john.tan\AppData\Local\Temp\ipykernel_14684\2230247851.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_upload_diff.rename(columns={'LOADING PLANT': 'LOADING'}, inplace=True)


,SAP Order No,LOADING,DELIVERY
0,12743533,SUMITOMO CHEMICAL ASIA PTE LTD,03/02/2023
1,12737629,MITSUI ELASTOMERS SINGAPORE PTE LTD,07/02/2023
2,5955534,CHAMPIONX SG SERVICE PTE LTD,02/02/2023
3,12734191,BASF SOUTH EAST ASIA PTE LTD,07/02/2023
4,12734189,BASF SOUTH EAST ASIA PTE LTD,27/01/2023
5,5955180,CHAMPIONX SG SERVICE PTE LTD,01/02/2023
6,12743534,SUMITOMO CHEMICAL ASIA PTE LTD,17/02/2023
7,12734192,BASF SOUTH EAST ASIA PTE LTD,09/02/2023
8,5944859,CHAMPIONX SG SERVICE PTE LTD,17/01/2023
9,12737634,MITSUI ELASTOMERS SINGAPORE PTE LTD,17/02/2023


In [117]:
#df3 = df2.reindex_like(df1)
#df1[(df3['a'].isin(df1['a'])) & (df1['b'] <= df3['b'])]
comp_6 = temp_upload_1.reindex_like(master)
master[(comp_6['Cust Ref'].isin(master['Cust Ref'])) & (master['PRODUCT'] <= comp_6['PRODUCT'])]
#master[np.logical_not(comp_6['Cust Ref'].isin(master['Cust Ref'])) & (master['DEPOT'] <= comp_6['DEPOT'])]

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
7,1003981242,6108316731,2023-01-16,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXAL 8 ROP VL,BASF 20 BANYAN AVENUE JURONG ISLAND,2023-01-16,5954173
19,1003982292,6108324472,2023-01-31,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI 701 AYER MERBAU ROAD,2023-01-31,12720816
20,1003982230,6108324473,2023-01-31,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI 701 AYER MERBAU ROAD,2023-01-31,12720817
21,1003979353,6108305780,2023-01-25,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXSOL HEXANE SP AP VL,ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD,2023-01-25,12726060
24,1003982065,6108322714,2023-01-30,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXSOL HEPTANE AP VL,MSD 50 TUAS WEST DRIVE,2023-01-30,12726480
25,1003980890,6108314619,2023-01-27,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXSOL HEXANE SP AP VL,THE POLYOLEFIN - PULAU AYER MERBAU \nSTREET 2 ...,2023-01-27,12727051
27,1003981261,6108317029,2023-01-20,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXAL 8 ROP VL,BASF 20 BANYAN AVENUE JURONG ISLAND,2023-01-20,12734187
28,0,0,2023-02-13,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXAL 8 ROP VL,BASF 20 BANYAN AVENUE JURONG ISLAND,2023-02-13,12734188
29,1003981271,6108317031,2023-01-27,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXAL 8 ROP VL,BASF 20 BANYAN AVENUE JURONG ISLAND,2023-01-27,12734189
31,1003984405,6108341829,2023-02-07,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXAL 8 ROP VL,BASF 20 BANYAN AVENUE JURONG ISLAND,2023-02-07,12734191


In [ ]:
import xlsxwriter #write to excel new transports 50

#writer1 = pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/temp_upload.xlsx", engine='xlsxwriter')
#temp_upload_1.to_excel(writer1, sheet_name='temp_upload') #write to excel with color
#worksheet1 = writer1.sheets['temp_upload']
#worksheet1.set_tab_color('blue')
#writer1.save()
#writer1.close()

#writer2 = pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/master.xlsx", engine='xlsxwriter')
#master.to_excel(writer2, sheet_name='master')
#worksheet2 = writer2.sheets['master']
#worksheet2.set_tab_color('purple')
#writer2.save()
#writer2.close()

with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/NewTransports_ChawanVopak.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer3:
    sheetName = 'NewTransports_' + datetime.now().strftime("%Y%m%d %H%M")
    temp_upload_1.to_excel(writer3, sheet_name=sheetName, engine='openpyxl')
    worksheet3 = writer3.sheets[sheetName]
    writer3.save()

with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/NewTransports_50.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer4:
    sheetName = 'Transporeon50_' + datetime.now().strftime("%Y%m%d %H%M")
    df_trans.to_excel(writer4, sheet_name=sheetName, engine='openpyxl')
    worksheet4 = writer4.sheets[sheetName]
    writer4.save()
  
print('\033[1m'+ '\033[34m' + 'Process completed')